## Install Lib API

In [ ]:
! pip install https://dnaink.jfrog.io/artifactory/dna-ink-pypi/model-fkeywords/0.1.0/model_fkeywords-0.1.0-py3-none-any.whl

In [ ]:
! python -m spacy download pt_core_news_sm

## Import libs

In [1]:
import pandas as pd
from api_model.nlsuper import NlExtractorProcess
from api_model.nlvisualization import NlVisualization

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/tmp/ipykernel_199/1571986624.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Variables

##### Description of process
        whats_process = 'complete'
            return: process all pipeline
        whats_process = 'partial'
            return: findkeywords and process bigrams
        whats_process = 'only_keywords'
            return: findkeywords   

In [2]:
filename = 'LIBERACAO_MARGEM_TESTE'
prefix = 'csv'
prefix_sep = '|'
column_text = 'TRANSCRICAO'
whats_process = 'complete'
#whats_process = 'partial'
#whats_process = 'only_keywords'
id_database = 'CONTACTID'
type_find = 'aproximado' # aproximado # fixo
activate_stopwords = 'sim' # não

In [3]:
text_finds = {
    'negatividade': ['a desejar', 'abrir uma reclamacao', 'absurd', 'aguento mais', 'boceta', 'bosta', 'brigaram', 'buceta', 'burocra', 'cansad', 'caralho', 'conflito', 'constrang', 'decepcao', 'decepcionad', 'quanta demora', 'que demora', 'tentando resolver', 'descontent', 'descrenca', 'descrente', 'desrespeit', 'fdp', 'filha da puta', 'filha de uma puta', 'filho da puta', 'filho de uma puta', 'frustracao', 'frustrad', 'humilhacao', 'humilhad', 'ignorad', 'ignoram', 'incompet', 'inferno', 'informa nada', 'injuria', 'ironia', 'ironic', 'irreponsaveis', 'irresponsabilidade', 'irresponsavel', 'ma vontade', 'mal a pior', 'mal atendid', 'mal educad', 'mal respondem', 'mal tratad', 'mau atendid', 'mau educad', 'mau respondem', 'mau tratad', 'merda', 'nao podemos arcar', 'nao posso arcar', 'nao sabe', 'ninguem resolve', 'ofenderam', 'ofendid', 'orrivel', 'pelo amor de deus', 'pessimo', 'pessimu', 'porra', 'poxa', 'pqp', 'puta que pariu', 'reclamacao', 'reclamar', 'ridicul', 'ruim', 'sabe nada', 'sem educacao', 'ta dificil', 'trata mal', 'trata mau', 'tratou mal', 'tratou mau', 'triste', 'vergonh', 'vagabund', 'raiva', 'detest', 'nao quero mais', 'odeio', 'safad', 'deus me livre', 'vergonh'],
    'reputacao': ['advogad', 'consumidor.gov', 'entrar com processo', 'facebook', 'instagram', 'judiciais', 'judicial', 'justica', 'meu direito', 'meus direito', 'ouvidoria', 'pequenas causas', 'processar voces', 'procon', 'reclame aqui', 'reclameaqui', 'twitter'],
    'rechamada': ['ainda nao', 'alguma posicao', 'ate agora', 'ate o momento', 'atraso', 'chamado aberto', 'consigo resolver', 'contato novamente', 'continuo sem acesso', 'de novo', 'demora', 'desde ontem', 'desde semana', 'diversas vezes', 'duas vezes', 'em andamento', 'entrei em contato', 'era pra ter', 'faz um mes', 'faz uma semana', 'ja abri', 'ja enviei', 'ja liguei', 'ja mandei', 'liguei pra central', 'mesmo erro', 'muitas vezes', 'nao chegou', 'nao e a primeira vez', 'ocorrencia aberta', 'posicionamento', 'segunda vez', 'sem sucesso', 'tentativa', 'terceira vez', 'todo dia', 'varias vezes', 'chamado aberto'],
    'satisfacao': ['adorei', 'adoro', 'ageis', 'agil', 'agilidade', 'amei', 'atendimento perfeito', 'bem atendid', 'carisma', 'diferenciad', 'educad', 'eficacia', 'eficas', 'eficaz', 'elogiar', 'elogio', 'excelente atendimento', 'feliz', 'gostei', 'impecavel', 'me ajudou', 'merito', 'motivacao', 'motivad', 'muito 10', 'muito bom', 'muito dez', 'nota 10', 'nota dez', 'otimo atendimento', 'parabenizar', 'parabens', 'pela ajuda', 'personalizad', 'preparad', 'prestativ', 'profissional', 'qualidade', 'rapidez', 'resolveu', 'satisfacao', 'satisfatori', 'satisfeit', 'solucionou', 'ate que enfim', 'ate que em fim'],
    'concorrencia': ['ifood', 'caju', 'vee', 'flash', 'bem', 'up', 'VR', 'ticket', 'alelo', 'swile'],
    'contencao': ['desculpa', 'desculpe', 'sinto muito', 'desculpe a demora', 'lamento', 'lamentamos', 'perdoa', 'perdao', 'pelo ocorrido', 'o ocorrido', 'fique tranquil'],
    'ocorrencia': ['chamado', 'ocorrencia', 'registrada com sucesso'],
    'direcionamento': ['central de atedimento', 'ligar na central', 'somente na central', 'sodexo.com', 'ligue na central'],
    'pedefacil': ['clique no', 'clique em', 'menu servicos', 'do menu', 'pede facil', 'clica no', 'clica em', 'diretamente no site', 'visualizar em', 'no menu', 'menu'],
    'ativo': ['analista pode entrar em contato', 'o analista entrara em contato', 'para te auxiliar na proposta', 'solicitei o contato', 'deixe seu contato']

          }

In [4]:
additional_stop_words = ['porque','bom','dia','tres','três','alo','alô']

In [5]:
interlocutor = {'CANAL': ['LEFT', 'RIGHT']}
response_time = 'STARTTIME'
format_data = '%d/%m/%Y %H:%M:%S|%d/%m/%Y %H:%M|%Y-%m-%d %H:%M:%S|%d-%m-%Y %H:%M|%d%m%Y %H:%M:%S|%d%b%Y:%H:%M:%S' # 03MAR2022:12:01:33

In [6]:
df = NlExtractorProcess.call_process(filename, prefix, prefix_sep,\
                                     column_text, whats_process,\
                                     text_finds, id_database, type_find,\
                                     additional_stop_words, activate_stopwords,\
                                    interlocutor, response_time, format_data)

[07/04 15:41:17] - [Info]:  Load CSV
[07/04 15:41:17] - [Info]:  read file /opt/dna/find-keywords/datalake/csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5002 entries, 0 to 5001
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CPF           5002 non-null   int64  
 1   DURATION      5002 non-null   int64  
 2   ENDTIME       5002 non-null   object 
 3   EXTERNALID    5002 non-null   object 
 4   ID            5002 non-null   int64  
 5   STARTTIME     5002 non-null   object 
 6   MONOCHANNEL   4 non-null      object 
 7   SEQUENCIA     4997 non-null   float64
 8   CANAL         4997 non-null   object 
 9   TRANSCRICAO   4997 non-null   object 
 10  AGENT         5002 non-null   object 
 11  WORKGROUP     5002 non-null   object 
 12  CATEGORIES    5002 non-null   object 
 13  TOPICS        4964 non-null   object 
 14  RECHAMADAS    0 non-null      float64
 15  URA_LOG       5002 non-null   int64  
 16  FIL

22/07/04 18:42:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


[07/04 15:42:42] - [Info]:  Finishing Process
[07/04 15:42:42] - [Info]:  created message order
Generating `message_order` column
root
 |-- CPF: string (nullable = true)
 |-- DURATION: string (nullable = true)
 |-- ENDTIME: string (nullable = true)
 |-- EXTERNALID: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- message_time: string (nullable = true)
 |-- MONOCHANNEL: string (nullable = true)
 |-- SEQUENCIA: string (nullable = true)
 |-- message_author: string (nullable = true)
 |-- message_content: string (nullable = true)
 |-- AGENT: string (nullable = true)
 |-- WORKGROUP: string (nullable = true)
 |-- CATEGORIES: string (nullable = true)
 |-- TOPICS: string (nullable = true)
 |-- RECHAMADAS: string (nullable = true)
 |-- URA_LOG: string (nullable = true)
 |-- FILA: string (nullable = true)
 |-- VQ: string (nullable = true)
 |-- CALLTYPE: string (nullable = true)
 |-- issue_id: string (nullable = true)
 |-- DATA_ARQUIVO: string (nullable = true)
 |-- negatividade_fin

[07/04 15:42:45] - [Debug]:  count rows after remove null values 4315
[07/04 15:42:45] - [Debug]:  created a new collect dict of interlocutor
[07/04 15:42:45] - [Debug]:  new collect dict {'message_author': ['LEFT', 'RIGHT']}
[07/04 15:42:45] - [Debug]:  created a original column
[07/04 15:42:46] - [Info]:  agroup all menssages for ticket
Grouping messages into one unique record
[07/04 15:42:46] - [Debug]:  Message Author Column: message_author
[07/04 15:42:46] - [Debug]:  Message content Column: message_content
[07/04 15:42:46] - [Debug]:  [], []
[07/04 15:42:46] - [Debug]:  separadores cliente e operador, {'message_author': ['LEFT', 'RIGHT']}
[07/04 15:42:46] - [Debug]:  Joining messages to all_messages
[07/04 15:42:46] - [Debug]:  Joining messages to all_messages_message_author_LEFT
[07/04 15:42:46] - [Debug]:  Joining messages to all_messages_message_author_RIGHT
[07/04 15:42:46] - [Debug]:  MAX columns: {'rechamada_findint', 'direcionamento_findint', 'pedefacil_findint', 'reputaca

[07/04 15:42:51] - [Info]:  numbers of rows agrouped 16
[07/04 15:42:51] - [Info]:  process bigrams and trigrams of column_text


22/07/04 18:43:38 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- issue_id: string (nullable = true)
 |-- RECHAMADAS: string (nullable = true)
 |-- MONOCHANNEL: string (nullable = true)
 |-- CALLTYPE: string (nullable = true)
 |-- has_attendant: integer (nullable = true)
 |-- CPF: string (nullable = true)
 |-- TOPICS: string (nullable = true)
 |-- AGENT: string (nullable = true)
 |-- WORKGROUP: string (nullable = true)
 |-- CATEGORIES: string (nullable = true)
 |-- message_author: string (nullable = true)
 |-- SEQUENCIA: string (nullable = true)
 |-- DURATION: string (nullable = true)
 |-- ENDTIME: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- numbers_words: string (nullable = true)
 |-- URA_LOG: string (nullable = true)
 |-- EXTERNALID: string (nullable = true)
 |-- VQ: string (nullable = true)
 |-- message_order: integer (nullable = true)
 |-- message_time: string (nullable = true)
 |-- FILA: string (nullable = true)
 |-- DATA_ARQUIVO: string (nullable = true)
 |-- rechamada_findint: string (nullable = true)
 |-- direcio

[07/04 15:44:11] - [Info]:  Finishing Process and Save csv File
[07/04 15:44:11] - [Info]:  save csf file
[07/04 15:44:11] - [Info]:  Finishing Process
[07/04 15:44:11] - [Info]:  deleting file: /opt/dna/find-keywords/datalake//LIBERACAO_MARGEM_TESTE_temp.csv
[07/04 15:44:11] - [Debug]:  file temp deleted


In [ ]:
df.head()

In [ ]:
df.count()

In [ ]:
filename = 'LIBERACAO_MARGEM_TESTE'
column_filter = '' # deixar esse campo vazio caso queira ver tudo
column_text = 'all_messages'
whats_process = 'trigram' # trigram or bigram

In [ ]:
NlVisualization.wordCloud_Topics(filename,column_filter,column_text,whats_process)

In [ ]:
NlVisualization.plot_10_most_common_words(filename,column_filter,column_text)